In [1]:
import pandas as pd

In [2]:
file_path = '../Cleaned_Data.xlsx'
sheets = pd.read_excel(file_path, sheet_name=None)

In [3]:
def add_composite_features(df):
    df['finishing'] = df.get('Goals', 0) + df.get('Shots per game', 0)
    df['creativity'] = df.get('Assists', 0) + df.get('Key passes per game', 0) + df.get('Through balls per game', 0) + df.get('Dribbles per Game', 0) + df.get('Fouled per Game', 0)
    df['distribution'] = df.get('Pass success percentage', 0) + df.get('Crosses', 0)  + df.get('Long balls per game', 0)
    df['defense'] = df.get('Interceptions per game', 0) + df.get('Clearances per game', 0) + df.get('Outfielder block per game', 0)
    df['duels'] = df.get('Aerials won per game', 0) - df.get('Dispossessed per game', 0) - df.get('Bad control per game', 0)

    composite_cols = ['finishing', 'creativity', 'distribution', 'defense', 'duels']
    df[composite_cols] = df[composite_cols].round(2)
    
    return df

In [4]:
# Apply composite feature engineering to each DataFrame
for position in sheets:
    sheets[position] = add_composite_features(sheets[position])

In [5]:
with pd.ExcelWriter('Composite_Features_Output.xlsx') as writer:
    for position, df in sheets.items():
        df.to_excel(writer, sheet_name=position, index=False)

In [12]:
# Load the sheets
file = 'Composite_Features_Output.xlsx'
sheets = pd.read_excel(file, sheet_name=None)

# Extract sheets
df = sheets['Data']
def_df = sheets['DEF']
mid_df = sheets['MID']
off_df = sheets['OFF']

# Create sets of player names for each position
def_players = set(def_df['Name'])
mid_players = set(mid_df['Name'])
off_players = set(off_df['Name'])

# Create new binary position columns in the general sheet
df['DEF'] = df['Name'].apply(lambda x: 1 if x in def_players else 0)
df['MID'] = df['Name'].apply(lambda x: 1 if x in mid_players else 0)
df['OFF'] = df['Name'].apply(lambda x: 1 if x in off_players else 0)
df.drop(columns=['Name', 'Position', 'Player Info '], errors='ignore', inplace=True)
# Save the updated general sheet

df.to_excel('General_with_Positions.xlsx', index=False)

In [ ]:
POSITION_FEATURE_WEIGHTS = {
    'OFF': {
        'finishing': 3,
        'creativity': 3,
        'distribution': 1,
        'defense': 1,
        'duels': 2
    },
    'MID': {
        'finishing': 2,
        'creativity': 3,
        'distribution': 3,
        'defense': 2,
        'duels': 2
    },
    'DEF': {
        'finishing': 1,
        'creativity': 1,
        'distribution': 2,
        'defense': 4,
        'duels': 3
    }
}

def compute_score(row):
    if row.get('off', 0) == 1:
        weights = POSITION_FEATURE_WEIGHTS['OFF']
    elif row.get('mid', 0) == 1:
        weights = POSITION_FEATURE_WEIGHTS['MID']
    elif row.get('def', 0) == 1:
        weights = POSITION_FEATURE_WEIGHTS['DEF']
    else:
        weights = {}

    score = sum(row.get(feat, 0) * weight for feat, weight in weights.items())
    return round(score, 2)


df = add_composite_features(df)
df['weighted_score'] = df.apply(compute_score, axis=1)